<a href="https://colab.research.google.com/github/abdulbaseet-zahir/Kurdish-next-word-predictor/blob/main/kurdish_next_word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import tensorflow

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding

from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical

from string import punctuation
import re
import pickle
import collections
import numpy as np
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/kurdish next word prediction/AsoSoft mini-Text Corpus- Version 1.0 (2018-12-10).txt", "r", encoding = "utf8") as f:
    lines = [line for line in f]

In [ ]:
%%time
raw_data = ""

for i in lines[:20]:
    raw_data = ' '. join(lines[:20])
    
raw_data = raw_data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

CPU times: user 1.07 ms, sys: 10 µs, total: 1.08 ms
Wall time: 1.1 ms


In [ ]:
lines[:2]

[' هەڵبەتە لەیەککات و لەیەک سەرزەمیندا دوو گۆڕانکاری مەزن و پڕ سەروەری دەبێتە مایەی هاتنەکایەی خۆشگوزەرانی و شادنوودی گەل و کۆمەلانی خەڵک ، ئەو دوو حاڵەتە پڕ سەروەرییەش ، یەکەمیان ئاشکراکردنی حکومەتی کابینەی شەشەمە بەپۆلێک عەقڵییەتی تازەو بەرنامەی تازەترەوە بۆ خزمەتکردن بە هەرێمی کوردستان ، دووەمیشیان گرێدانی پلینۆمی ( ی.ن.ک ) بۆ دووبارە نوێبوونەوە و خوێنی گەرم بەبەرداکردنەوەی حیزبێک کەسالانێکی زۆر بەرپرسیارێتی لەئەستۆدایە لەهەرێمی کوردستان . گرفتێکی گەورە سەبارەت بەکابینەی شەشەم لەگەڵ پیرۆزباییدا لەگۆشەنیگای فەرمانبەر و کارگوزارانی فەرمانگەکانی هەرێمی کوردستانەوە بۆ ئەمجارەیان گۆشەنیگای خۆم دەخەمە بەر تیشکی ڕەخنەو پێشنیار ، هەڵبەتە ئەگەر بڕوانیتە چینی فەرمانبەران و کارگوزارانی فەرمانگەکانی حکومەتی هەرێمی جەنجاڵی و ئاپۆرایەکی زۆرو بێ سیستمی گونجاو دەبینین کە دەکرێت حکومەتی هەرێم بەشوێن سیستمێکی کاردا بگەڕێت بۆئەوەی لەم هەلایە دەربازی بێت ، ئەمڕۆ بەحوکمی کەڵەکەبوونی فەرمانبەرێکی زۆر لەئەنجامی دامەزراندنی بێپلان و بەرنامەی گشت ساڵەکانی بەر لەئێستا تووشی هەلاوسان و بێکاری هاتووین ، کە بۆخ

In [ ]:
print('Number of articls ->',len(lines))
print('Number of words ->',len(raw_data.split()))
print('Number of leters ->',len(raw_data))

Number of articls -> 7108
Number of words -> 18293
Number of leters -> 119087


In [ ]:
new_data = re.sub(r'[^\w\s]','', raw_data)

In [ ]:
# %%time
# z = []
# for i in new_data.split():
#     if i not in z:
#         z.append(i)

In [ ]:
%%time
z = collections.OrderedDict.fromkeys(new_data.split()).keys()

Wall time: 4.01 ms


In [ ]:
len(z)

6783

In [ ]:
word_data = ' '.join(z)

In [ ]:
print('Number of uniq words ->',len(word_data.split()))

Number of uniq words -> 6783


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([new_data])

In [ ]:
tokenizer.index_word.get(5)

'ئەم'

In [ ]:
word_dicts = tokenizer.index_word
keys = [k for k in word_dicts.keys()]
vals = [v for v in word_dicts.values()]

In [ ]:
with open("word_tokenizer1.txt", "w", encoding = "utf8") as output:
    for word in vals:
        output.writelines(word+"\n")

In [ ]:
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

In [ ]:
sequence_data = tokenizer.texts_to_sequences([new_data])[0]

In [ ]:
sequence_data[:10]

[410, 1996, 1, 326, 1997, 85, 1091, 1092, 1, 183]

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

6784

In [ ]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)

sequences = np.array(sequences)

In [ ]:
X1 = []
y1 = []

for i in sequences:
    X1.append(i[0])
    y1.append(i[1])
    
X = np.array(X1)
y = np.array(y1)

In [ ]:
y

array([1996,    1,  326, ..., 6782,  565, 6783])

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
model = tensorflow.keras.models.Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             67840     
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 6784)              6790784   
Total params: 19,907,624
Trainable params: 19,907,624
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import time
import datetime

current_time = str(int(time.time()))
current_time2 = '_'.join(str(time.asctime()).split())
current_time3 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint("nextword2.h5", monitor='accuracy', verbose=1, save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

tensorboard_Visualization = TensorBoard('logs\\'+current_time3, histogram_freq=1)

from tensorflow.keras.optimizers import Adam
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001), metrics=['accuracy'])

model.fit(X, y, epochs=150, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Train on 16613 samples
Epoch 1/150
16576/16613 [============================>.] - ETA: 0s - loss: 8.2856 - accuracy: 0.0633
Epoch 00001: accuracy improved from -inf to 0.06320, saving model to nextword2.h5
16613/16613 [==============================] - 31s 2ms/sample - loss: 8.2857 - accuracy: 0.0632
Epoch 2/150
16576/16613 [============================>.] - ETA: 0s - loss: 7.6885 - accuracy: 0.0640
Epoch 00002: accuracy improved from 0.06320 to 0.06417, saving model to nextword2.h5
16613/16613 [==============================] - 30s 2ms/sample - loss: 7.6880 - accuracy: 0.0642
Epoch 3/150
16576/16613 [============================>.] - ETA: 0s - loss: 7.4331 - accuracy: 0.0642
Epoch 00003: accuracy improved from 0.06417 to 0.06441, saving model to nextword2.h5
16613/16613 [==============================] - 34s 2ms/sample - loss: 7.4328 - accuracy: 0.0644
Epoch 4/150
16576/16613 [============================>.] - ETA: 0s - loss: 7.3207 - accuracy: 0.0644
Epoch 00004: accuracy did not imp

16576/16613 [============================>.] - ETA: 0s - loss: 3.9624 - accuracy: 0.2221
Epoch 00030: accuracy improved from 0.21417 to 0.22206, saving model to nextword2.h5
16613/16613 [==============================] - 32s 2ms/sample - loss: 3.9636 - accuracy: 0.2221
Epoch 31/150
16576/16613 [============================>.] - ETA: 0s - loss: 3.8742 - accuracy: 0.2343
Epoch 00031: accuracy improved from 0.22206 to 0.23421, saving model to nextword2.h5
16613/16613 [==============================] - 29s 2ms/sample - loss: 3.8750 - accuracy: 0.2342
Epoch 32/150
16576/16613 [============================>.] - ETA: 0s - loss: 3.7909 - accuracy: 0.2404
Epoch 00032: accuracy improved from 0.23421 to 0.24029, saving model to nextword2.h5
16613/16613 [==============================] - 28s 2ms/sample - loss: 3.7921 - accuracy: 0.2403
Epoch 33/150
16576/16613 [============================>.] - ETA: 0s - loss: 3.7195 - accuracy: 0.2433
Epoch 00033: accuracy improved from 0.24029 to 0.24336, saving

Epoch 90/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.3738 - accuracy: 0.3967
Epoch 00090: accuracy improved from 0.39559 to 0.39662, saving model to nextword2.h5
16613/16613 [==============================] - 41s 2ms/sample - loss: 2.3747 - accuracy: 0.3966
Epoch 91/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.3610 - accuracy: 0.3967
Epoch 00091: accuracy improved from 0.39662 to 0.39686, saving model to nextword2.h5
16613/16613 [==============================] - 34s 2ms/sample - loss: 2.3611 - accuracy: 0.3969
Epoch 92/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.3541 - accuracy: 0.3989
Epoch 00092: accuracy improved from 0.39686 to 0.39872, saving model to nextword2.h5
16613/16613 [==============================] - 29s 2ms/sample - loss: 2.3548 - accuracy: 0.3987
Epoch 93/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.3544 - accuracy: 0.3959
Epoch 00093: accuracy did not improve from 0.3987

16613/16613 [==============================] - 28s 2ms/sample - loss: 2.1940 - accuracy: 0.4044
Epoch 122/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.1914 - accuracy: 0.4047
Epoch 00122: accuracy improved from 0.40438 to 0.40474, saving model to nextword2.h5
16613/16613 [==============================] - 27s 2ms/sample - loss: 2.1910 - accuracy: 0.4047
Epoch 123/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.1988 - accuracy: 0.4008
Epoch 00123: accuracy did not improve from 0.40474
16613/16613 [==============================] - 26s 2ms/sample - loss: 2.1990 - accuracy: 0.4008
Epoch 124/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.1864 - accuracy: 0.4028
Epoch 00124: accuracy did not improve from 0.40474
16613/16613 [==============================] - 26s 2ms/sample - loss: 2.1878 - accuracy: 0.4026
Epoch 125/150
16576/16613 [============================>.] - ETA: 0s - loss: 2.1764 - accuracy: 0.4037
Epoch 00125: accu

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

string_word = 'کەناکرێت هەروا بەئاسانی تێیان بڕوانین بەڵکو بۆبکرێت خەڵکی شیاویان بۆبدۆزرێتەوە بەلام وەخت لەمیزاج تەکەتول دووربکەوینەوە هەنگاونان لێرەدا ئەوپەڕی دیموکراتیەت ئازادی بەخۆوە ببینێت کەسی شیاو بەدەنگی بچێتە جێیە بەتالانە ئەوەش لەبەرچاو بگرین دەیان سەرکردەی شەهیدبوون چەندین جار ئەوە هاتووە شوێنی سەرکردەکانی چۆڵ بن دواتر پڕبکرێنەوە لەزۆربەی کاتەکان بەرزکردنەوەی کەسێک یان کەسانێک بۆپڕکردنەوەی شوێنە بەتاڵەکان کراوە بۆیە زۆرجار دەرنەچووە کەچاوەڕوان واتە بەرزکردنەوە کەمتر لە خۆدەگرێت لەوەی هەڵبژاردنی پەیڕەوکردنی سزاو پاداشت گۆڕانە بەخۆیەوە دەبینێت دەستبەکار کەبەشێکی گرنگی کابینەیە پێکدەهێنێ گرێ دەدات بۆرێکخستنەوەی ماڵی یەکێتی لەگشت لایەن کەسەکان ڕووداوە بەگرنگ بخوێنێتەوە پلینۆمیش تەنیا سەرخستن جێ پڕکردنەوەو جێگۆڕکێ نەبێت بۆپەیڕەوکردنی پاداشتیش کەکاری خراپیان کردووە باشیان بگیرێت کەئەو کەسانەی نەیان توانیوە کارەکانیان جێبەجێبکەن بکەن بۆهەڤالانی هاتنەپێشەوە قولایی سیاسییەکان دوێنێ 31 10 2009 سێیەمین دانیشتنی یەکێتیی نیشتمانیی لەتەلاری هونەر لەسلێمانی بەڕێوەچوو تێیدا ڕاپۆرتی لیژنەی ناوچەکانی لەلایەن ڕزگار عەلی خوێنرایەوە ئاماژەیکرد بۆئەو ناوچانە کەپارێزگای کەرکوک سنووری نەینەواو دیالەو سەلاحەددین واست بەغدا دەگرێتەوە مێژووی سیاسەتی مەترسیداری تەعریب تەهجیرو دابڕاندنی ناوچانەی خستەڕوو پەنجەشی خستەسەر کێشەکانی دانیشتووانی بەدەست زوڵمی یاسایی ئەمنی ئابوورییەوە دەناڵێنن ئاسانکاری کێشەکان گوتیشی ماددەی 140 ی دەستوور ماددەیەکی دەستووریی UN گەڕایەوەسەر پارێزگای هەولێر تێبینی ڕەخنەکانی ئەندامان لەپێناوی زیاتر دەوڵەمەندکردنی کەگفتوگۆیەکی هەمەلایەنەی لەبارەوە کراو پەرۆشیی بەرجەستەدەکرد بەگەڕانەوەی جێناکۆک جێبەجێکردنی دەستووری جێناکۆکی لەوبارەیەشەوە مەلا بەختیار ئەندامێکی خانەقین خستنەڕووی هۆکارەکانی سستی داوایکرد شەڕی مەرکەزی ناکۆکییەکان کەشەڕی کورد لەدەرەوەی شارەکانی کێشە بەخزمەتکردن خزمەتگوزاری ئەولەویەت بدات بەسنووری وەزارەتێکی کارەی نەکرد لەبەشەکەی متمانەی لێوەربگیرێتەوە شێخ جەعفەر مستەفاش وێڕای پڕۆژەیەک پاراستنی ئاسایشی دووپاتیکردەوە کەئەگەر پڕۆژەیە جێبەجێنەکرا پێشمەرگە شوێنێک لەو دەمێنێتەوەو هیچ ناکات پاشان بەتێکڕای دەنگ پەسەندکرا سکالاو گازندەکان پەسەندکردنی حاکم قادر حەمەجان عەزیزەوە کەچەندین خاڵی تێدا خرابووەڕوو هەم وەکو گازەندەو ڕەخنە ڕاسپاردە زیندووێتی ژیانی ڕێکخراوەیی خالانەدا پابەندبوون بەپەیڕەوی ناوخۆی نەهێشتنی ناعەدالەتی ڕێگەگرتن لەزیادەڕۆیی دەسەلاتی هەندێک لەلێپرسراوان دووپاتکرابووەوە پێشنیازی گرنگ واقیعیش خرانەڕوو کەهەمووی بەئاراستەی بەهێزکردن گوڕوتین بەخشین بوو ئاماژەش کرابوو بەوەی کەزیاتر بایەخ بەژنان پێگەی ژن گەنجان کەسوکاری شەهیدان ئەنفالکراو کەمئەندامانی سەنگەر بدرێت یاداشتکرا کەیاسای کادیران ڕێکبخرێت بنەماو ڕێساکان کاریش بەپاداشت سزا ڕوونکرابووەوە کەسنوورێک دەستوەردانی لێپرسراوان لەبازرگانی کۆکردنەوەی سامان دابنرێت بنبڕکردنی دەستەگەری دەربڕینی وردی موناقەشەی دڵسۆزانە ڕەخنەی بنیاتنەر دروستکەری بەهێز هەروەها بەرلەوەی لیژنەکە بخرێتە دەنگدانەوە کۆسرەت ڕەسوڵ داوای چەند دەقیقەیەک دەرفەتی قسەکردنی کردو ڕێگەپێدانی گوتەیەکی پێشکەشکرد تەکەتولی دووپاتکردەوە داواشیکرد بەزووترین کات نەزاهە پێکبهێنرێت زانینی سەروەت سامانی داواشی لەئەندامانی ڕیزەکانی ڕێکخستن کرد کەیەکێتیی بپارێزن بەناوی سەرکردایەتیشەوە بەڵێنیدا کەپابەندی گفتانەبن کەدراوەو دەدرێت گوتی جێبەجێنەکران لەکۆنگرەدا سەرکردایەتییە هەڵمەبژێرن خرایە دەنگی پەیامی لەڕاگەیاندنی بڕیاری پێکهێنانی ڕاگەیاندن پێکهات لەوادەی دیاریکراوی خۆیدا کۆ بووەوە لەسێیەم ڕۆژی پلینۆمدا ئازاد جندیانی ڕاپۆرتەکەی خوێندەوە کۆنفرانسەی ڕاگەیاندنی لەناوەڕاستی مانگی ئۆکتۆبەردا بەسترا پێشخستن کرانەوەی تێبینییەکانی لیژنەکەی نەبوونی پەیام کەپەرشوبلاوی لەڕاگەیاندنەکاندا دروستکردووە کورتخایەن درێژخایەن گەشەپێدانی لەوبارەیەوە ڕوونکراوانەی لەنێو ڕاپۆرتەکەدا لەلیژنەکەدا پێشکەشیکردبوو ڕوونیکردەوە بەرجەستەیە پەیامیشی گەڕاندەوە لەمەکتەبی سیاسیدا درێژخایەنیش پلانە لەهەموو مەکتەب ئۆرگانەکانی کورستاندا پەیڕەوبکرێت ڕاو بۆچوونی تێروتەسەل بەزۆرینەی بایەخدانی بەکەسوکاری لەدانیشتنی دوانیوەڕۆی سێیەم لیژنەیەکی پەسەندکران لەدانیشتنەکەدا بۆردومانی کیمیایی ئەنفال وارسەکانیان سەنگەرو زیندانیانی پێشکەشکران عەبدولکەریم هەڵەدنی خوێندیەوە پڕۆژە لەخۆگرتبوو لەبەرژەوەندیی کەیسانە گرنگیدان بەوارسی قوربانیانی چەکی تێکڕای خانووبەرە خوێندنی کوڕان کچانی شەهید وارسی تاچڕکردنەوەی هەوڵەکان کۆمەڵکوژی لەجیهاندا بەجینۆساید بناسرێت دروستکردنی مۆنیومێنت هێنانەوەی ڕوفاتی ئەنفالکراوەکان پێشنیازو لەڕاپۆرتی لیژنەکەدا قەرەبووبکرێنەوەو ژیانێکی شایستەیان دابینبکرێت کەبارتەقای قوربانیەکانیان تەئکیدکرایەوە کەدەبێت تێکۆشەرانی کۆچکردوو پۆلێنبکرێن تاوەکو ئەوانیش ژیانیان پڕۆژەکە لەبارەوەکراو بەئازادی ڕاشکاوی تێبینییەکانیان داکۆکییان لەمافی ڕەخنەش کەموکوڕیانە گیرا کەدەرهەق بەلایەنە پەیوەستەکانی کەیسەوە لەکۆتاییشدا مەکتەبەکانی لەڕاپۆرتەکەدا ئاماژە کرا ڕێکخراوە دێموکراتییەکان کەهەوڵبدات یەکخستنەوەی ڕێکخراوو کۆمەڵەکان گوێدانە ئینتیمای هاوئاهەنگیی نێوان داوایەکی لیژنەیەبوو لەکەموکورتییەکان داوایکردبوو کارئاسانی چاودێری ئۆرگانەکان بەسیستمی دامەزراوەیی لەکەموکورتی لەمەکتەبەکان مەکتەبەکان پەیڕەوی ناوخۆیان'
list_words = string_word.split()

model = load_model('nextword2.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

for word in list_words:
    
    try:
        seq = np.array(tokenizer.texts_to_sequences([word])[0])
        predd = model.predict_classes(seq)

        for key, value in tokenizer.word_index.items():
                    if value == predd:
                        predicted_word = key
                        break
        print('seq ->',str(tokenizer.index_word[int(seq)]))
        print('key ->',key)
        print('value ->',value)
        print('predicred ->', predicted_word)
        print('')
    except: continue
        
        

seq -> کەناکرێت
key -> هەروا
value -> 564
predicred -> هەروا

seq -> هەروا
key -> ئاسان
value -> 1422
predicred -> ئاسان

seq -> بەئاسانی
key -> تێیان
value -> 2146
predicred -> تێیان

seq -> تێیان
key -> بڕوانین
value -> 2147
predicred -> بڕوانین

seq -> بڕوانین
key -> بەڵکو
value -> 97
predicred -> بەڵکو

seq -> بەڵکو
key -> هەوڵیداوە
value -> 891
predicred -> هەوڵیداوە

seq -> بۆبکرێت
key -> و
value -> 1
predicred -> و

seq -> خەڵکی
key -> ساکار
value -> 3311
predicred -> ساکار

seq -> شیاویان
key -> بۆبدۆزرێتەوە
value -> 2150
predicred -> بۆبدۆزرێتەوە

seq -> بۆبدۆزرێتەوە
key -> بەلام
value -> 6
predicred -> بەلام

seq -> بەلام
key -> ئەو
value -> 4
predicred -> ئەو

seq -> وەخت
key -> لەمیزاج
value -> 2151
predicred -> لەمیزاج

seq -> لەمیزاج
key -> و
value -> 1
predicred -> و

seq -> تەکەتول
key -> و
value -> 1
predicred -> و

seq -> دووربکەوینەوە
key -> بۆئەوەی
value -> 93
predicred -> بۆئەوەی

seq -> هەنگاونان
key -> لێرەدا
value -> 565
predicred -> لێرەدا

seq -> لێرەدا
key ->

seq -> ناوچانە
key -> و
value -> 1
predicred -> و

seq -> کەپارێزگای
key -> کەرکوک
value -> 49
predicred -> کەرکوک

seq -> کەرکوک
key -> و
value -> 1
predicred -> و

seq -> سنووری
key -> مێژوویی
value -> 732
predicred -> مێژوویی

seq -> نەینەواو
key -> دیالەو
value -> 2207
predicred -> دیالەو

seq -> دیالەو
key -> سەلاحەددین
value -> 2208
predicred -> سەلاحەددین

seq -> سەلاحەددین
key -> و
value -> 1
predicred -> و

seq -> واست
key -> و
value -> 1
predicred -> و

seq -> بەغدا
key -> دەگرێتەوە
value -> 1162
predicred -> دەگرێتەوە

seq -> دەگرێتەوە
key -> بڕوایەکی
value -> 925
predicred -> بڕوایەکی

seq -> مێژووی
key -> سیاسەتی
value -> 232
predicred -> سیاسەتی

seq -> سیاسەتی
key -> مەترسیداری
value -> 2210
predicred -> مەترسیداری

seq -> مەترسیداری
key -> تەعریب
value -> 572
predicred -> تەعریب

seq -> تەعریب
key -> من
value -> 70
predicred -> من

seq -> تەهجیرو
key -> دابڕاندنی
value -> 2212
predicred -> دابڕاندنی

seq -> دابڕاندنی
key -> ئەو
value -> 4
predicred -> ئەو

seq -> ناوچان

seq -> کەگفتوگۆیەکی
key -> جددی
value -> 576
predicred -> جددی

seq -> هەمەلایەنەی
key -> لەبارەوە
value -> 2264
predicred -> لەبارەوە

seq -> لەبارەوە
key -> کراو
value -> 2265
predicred -> کراو

seq -> کراو
key -> پەرۆشیی
value -> 1186
predicred -> پەرۆشیی

seq -> پەرۆشیی
key -> یەکێتی
value -> 44
predicred -> یەکێتی

seq -> بەرجەستەدەکرد
key -> بەگەڕانەوەی
value -> 2267
predicred -> بەگەڕانەوەی

seq -> بەگەڕانەوەی
key -> ناوچەکانی
value -> 283
predicred -> ناوچەکانی

seq -> جێناکۆک
key -> و
value -> 1
predicred -> و

seq -> جێبەجێکردنی
key -> بەڵێنەکان
value -> 1068
predicred -> بەڵێنەکان

seq -> دەستووری
key -> بیت
value -> 1859
predicred -> بیت

seq -> جێناکۆکی
key -> لەوبارەیەشەوە
value -> 1188
predicred -> لەوبارەیەشەوە

seq -> لەوبارەیەشەوە
key -> مەلا
value -> 346
predicred -> مەلا

seq -> مەلا
key -> بەختیار
value -> 1189
predicred -> بەختیار

seq -> بەختیار
key -> لەبارەی
value -> 143
predicred -> لەبارەی

seq -> ئەندامێکی
key -> پلینۆم
value -> 137
predicred -> پلینۆم

seq 

seq -> لەبازرگانی
key -> و
value -> 1
predicred -> و

seq -> کۆکردنەوەی
key -> گوێز
value -> 153
predicred -> گوێز

seq -> سامان
key -> دابنرێت
value -> 1211
predicred -> دابنرێت

seq -> دابنرێت
key -> بنبڕکردنی
value -> 799
predicred -> بنبڕکردنی

seq -> بنبڕکردنی
key -> دەستەگەری
value -> 800
predicred -> دەستەگەری

seq -> دەستەگەری
key -> و
value -> 1
predicred -> و

seq -> دەربڕینی
key -> سەرنج
value -> 284
predicred -> سەرنج

seq -> وردی
key -> ئەندامانی
value -> 193
predicred -> ئەندامانی

seq -> موناقەشەی
key -> جددی
value -> 576
predicred -> جددی

seq -> دڵسۆزانە
key -> درەنگ
value -> 6316
predicred -> درەنگ

seq -> ڕەخنەی
key -> دڵسۆزانی
value -> 1950
predicred -> دڵسۆزانی

seq -> بنیاتنەر
key -> و
value -> 1
predicred -> و

seq -> دروستکەری
key -> بەهێز
value -> 802
predicred -> بەهێز

seq -> بەهێز
key -> هەروەها
value -> 99
predicred -> هەروەها

seq -> هەروەها
key -> هەردی
value -> 1488
predicred -> هەردی

seq -> بەرلەوەی
key -> ڕاپۆرتی
value -> 167
predicred -> ڕاپۆرتی

seq

seq -> کیمیایی
key -> و
value -> 1
predicred -> و

seq -> ئەنفال
key -> و
value -> 1
predicred -> و

seq -> وارسەکانیان
key -> و
value -> 1
predicred -> و

seq -> سەنگەرو
key -> زیندانیانی
value -> 811
predicred -> زیندانیانی

seq -> زیندانیانی
key -> سیاسی
value -> 15
predicred -> سیاسی

seq -> پێشکەشکران
key -> کە
value -> 3
predicred -> کە

seq -> عەبدولکەریم
key -> هەڵەدنی
value -> 2403
predicred -> هەڵەدنی

seq -> هەڵەدنی
key -> خوێندیەوە
value -> 2404
predicred -> خوێندیەوە

seq -> خوێندیەوە
key -> ڕاپۆرتەکە
value -> 194
predicred -> ڕاپۆرتەکە

seq -> پڕۆژە
key -> ئەتۆمییەکەی
value -> 4844
predicred -> ئەتۆمییەکەی

seq -> لەخۆگرتبوو
key -> لەبەرژەوەندیی
value -> 2406
predicred -> لەبەرژەوەندیی

seq -> لەبەرژەوەندیی
key -> ئەو
value -> 4
predicred -> ئەو

seq -> کەیسانە
key -> لەوانە
value -> 436
predicred -> لەوانە

seq -> گرنگیدان
key -> بەوارسی
value -> 2409
predicred -> بەوارسی

seq -> بەوارسی
key -> ئەنفال
value -> 602
predicred -> ئەنفال

seq -> قوربانیانی
key -> چەکی
value 

In [ ]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer

model = load_model('nextword2.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
#         print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                
                break
        
        print(predicted_word)
        
        return predicted_word

In [ ]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "at the dull"
# text2 = "collection of textile"
# text3 = "what a strenuous"
# text4 = "stop the script"

while(True):

    text = input("Enter your line: ")
    
    if text == "stop":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue

Enter your line: هەروا
Enter your line: stop
Ending The Program.....


In [ ]:
model2 = model.save('my_model2.h5', )

In [ ]:
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

In [ ]:
seq = np.array(tokenizer.texts_to_sequences(['و'])[0])
predd = model.predict(seq)
predd

In [ ]:
for key, value in tokenizer.word_index.items():
            if (value == predd).all():
                predicted_word = key
                break
        
print(predicted_word)

In [ ]:
def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
#         print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        print(predicted_word)
        return predicted_word

In [ ]:
"""
    We are testing our model and we will run the model
    until the user decides to stop the script.
    While the script is running we try and check if 
    the prediction can be made on the text. If no
    prediction can be made we just continue.

"""

# text1 = "at the dull"
# text2 = "collection of textile"
# text3 = "what a strenuous"
# text4 = "stop the script"

while(True):

    text = input("Enter your line: ")
    
    if text == "stop the script":
        print("Ending The Program.....")
        break
    
    else:
        try:
            text = text.split(" ")
            text = text[-1]

            text = ''.join(text)
            Predict_Next_Words(model, tokenizer, text)
            
        except:
            continue